In [1]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
model = BertForMaskedLM.from_pretrained("bert-base-chinese")

/Users/yukun/Desktop/mda/mdapaper/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# 示例文本列表（你应替换为你真实的 MDA 文本数据）
mda_texts = [
    "公司2023年度实现营业收入增长，主要受益于主业扩张...",
    "2022年本公司在战略布局方面进行了调整，提升风险管理...",
    "受外部政策影响，公司整体毛利率出现下滑..."
]

from datasets import Dataset

# 构建 HuggingFace 数据集
dataset = Dataset.from_dict({'text': mda_texts})

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# 应用分词函数
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map: 100%|██████████| 3/3 [00:00<00:00, 12.45 examples/s]


In [5]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15  # 随机遮盖 15% token
)

In [18]:
import os
import torch
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

# 设置 CSV 文件路径
csv_file_path = "/Users/yukun/Desktop/mda/主观分析预处理/主观分析预处理结果/519GLM-Z1-9B-0414.env.csv"  # 替换为你的 CSV 文件路径

# 检查文件是否存在
if not os.path.exists(csv_file_path):
    raise FileNotFoundError(f"文件 {csv_file_path} 不存在，请检查路径是否正确！")

# 使用 pandas 读取 CSV 文件
df = pd.read_csv(csv_file_path)

# 打印列名
print("CSV 文件的列名：", df.columns)

# 将每一列的内容合并为一个整体的文本数据
# 假设每一行的每一列都是一个文本片段，我们将它们拼接起来
def combine_columns(row):
    return " ".join([str(cell) for cell in row])

# 应用 combine_columns 函数到每一行
df["combined_text"] = df.apply(combine_columns, axis=1)

# 打印合并后的前几行数据
print("合并后的前几行数据：")
print(df["combined_text"].head())

# 将数据转换为 Hugging Face Dataset 格式
dataset = Dataset.from_pandas(df[["combined_text"]])  # 只保留合并后的文本列

# 加载预训练的分词器
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

# 定义分词函数
def tokenize_function(examples):
    return tokenizer(examples["combined_text"], padding="max_length", truncation=True, max_length=512)

# 应用分词函数到数据集
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 定义数据组合器
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# 定义训练参数
training_args = TrainingArguments(
    output_dir="./mda-mlm",
    save_steps=500,
    logging_steps=100,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False,
    fp16=True if torch.cuda.is_available() else False
)

# 加载预训练模型
model = BertForMaskedLM.from_pretrained("bert-base-chinese")

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 开始训练
trainer.train()


CSV 文件的列名： Index(['文件名', '董事会成员变动', '董秘变更', '高级管理层变更', '重大资产重组', '控股股东/实际控制人变更',
       '子公司或重要分支机构重大变化', '对财务状况的分析', '营业收入增长情况', '净利润变动分析', '毛利率变动分析',
       '销售费用率变化', '管理费用控制情况', '经营性现金流变化趋势', '应收账款变化与风险', '资产负债结构调整',
       '利润质量分析（是否依赖非经常性损益）', '核心竞争力分析（品牌、成本、效率等）', '市场拓展与业务布局进展',
       '区域/产品结构优化情况', '供应链/渠道建设进展', '战略合作与外部联盟情况', '投资项目执行及回报分析',
       '企业文化建设与团队稳定性', '对标管理与精细化运营成果', '技术创新与研发投入', '新产品/新技术推出情况', '研发费用占比变化',
       '专利数量与知识产权布局', '智能制造/数字化转型进展', '行业竞争格局分析', '行业发展趋势与机会识别', '国家政策影响与公司应对',
       '未来经营计划与增长目标', '订单/客户/签约情况展望', '海外市场/出口业务计划', '对经济周期或宏观环境的判断',
       '面临的主要风险因素及变化', '政策/监管风险与公司响应', '环保合规/碳排放压力', '土地/原材料/人工成本波动风险',
       '资金链紧张预警', '核心客户集中度及流失风险', '可持续发展战略与目标', '社会责任履行与公益活动', '节能降耗/绿色生产措施'],
      dtype='object')
合并后的前几行数据：
0    2429-兆驰股份-2021.txt 未提及 未提及 未提及 2021年11月出售兆驰供应链...
1    895-双汇发展-2021.txt 未提及 未提及 未提及 未提及 未提及 未提及 1. 营...
2    2475-立讯精密-2021.txt 未提及 未提及 未提及 未提及 未提及 立铠精密科技（...
3    600566-济川药业-2021.txt 未提及 未提及 未提及 未提及 未提及 未提及 1...
4    963-华东医药-

/Users/yukun/Desktop/mda/mdapaper/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`combined_text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [16]:
import os
import torch
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

# 设置 CSV 文件路径
csv_file_path = "/Users/yukun/Desktop/mda/主观分析预处理/主观分析预处理结果/519GLM-Z1-9B-0414.env.csv"  # 替换为你的 CSV 文件路径

# 检查文件是否存在
if not os.path.exists(csv_file_path):
    raise FileNotFoundError(f"文件 {csv_file_path} 不存在，请检查路径是否正确！")

# 使用 pandas 读取 CSV 文件
df = pd.read_csv(csv_file_path)

# 检查数据
print("CSV 文件中的前几行数据：")
print(df.head())

# 确保 'text' 列存在
if "text" not in df.columns:
    raise ValueError("CSV 文件中没有找到 'text' 列，请检查列名是否正确！")

# 将数据转换为 Hugging Face Dataset 格式
dataset = Dataset.from_pandas(df)

# 加载预训练的分词器
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

# 定义分词函数
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# 应用分词函数到数据集
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 定义数据组合器
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# 定义训练参数
training_args = TrainingArguments(
    output_dir="./mda-mlm",
    save_steps=500,
    logging_steps=100,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False,
    fp16=True if torch.cuda.is_available() else False
)

# 加载预训练模型
model = BertForMaskedLM.from_pretrained("bert-base-chinese")

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 开始训练
trainer.train()


CSV 文件中的前几行数据：
                    文件名 董事会成员变动 董秘变更 高级管理层变更  \
0    2429-兆驰股份-2021.txt     未提及  未提及     未提及   
1     895-双汇发展-2021.txt     未提及  未提及     未提及   
2    2475-立讯精密-2021.txt     未提及  未提及     未提及   
3  600566-济川药业-2021.txt     未提及  未提及     未提及   
4     963-华东医药-2021.txt     未提及  未提及     未提及   

                                              重大资产重组 控股股东/实际控制人变更  \
0  2021年11月出售兆驰供应链85.22%股权（交易价300万元），导致资产减值准备计提18...          未提及   
1                                                未提及          未提及   
2                                                未提及          未提及   
3                                                未提及          未提及   
4           收购浙江道尔生物75%股权、安徽华昌高科100%股权；成立湖北美琪健康合资公司。          未提及   

                                      子公司或重要分支机构重大变化  \
0  2021年新增深圳市兆驰晶显技术、东莞兆驰智能、深圳市兆驰数码软件三家子公司，均未对业绩产生...   
1                                                未提及   
2  立铠精密科技（盐城）有限公司被收购并纳入合并范围，负责智能可穿戴设备生产；与奇瑞集团合资成立...   
3                                                未提及   
4

ValueError: CSV 文件中没有找到 'text' 列，请检查列名是否正确！